# WhisperTS joining JSON to SRT

In [17]:
import re,codecs,unidecode
import pandas as pd
import networkx as nx
import matplotlib.pylab as p

import json,glob,os
nMachine=os.uname()[1]
if "iMac" in nMachine:
    nMachine="iMac"
elif "MacBook" in nMachine:
    nMachine="MacBook"
print(nMachine)

In [18]:
repJson="/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/%s/Json/"%nMachine
segments=len(glob.glob(repJson+"*.json"))
segments

8

In [19]:
json1=repJson+"extrait-00.wav.json"
with open(json1) as inFile:
    whisper1=json.load(inFile)

### Gestion des timestamps

In [20]:
def second2timestamp(seconds):
    sHours=int(seconds/3600)
    sMinutes=int((seconds-sHours*3600)/60)
    sSeconds=int((seconds-sHours*3600-sMinutes*60))
    sMilliseconds=int((seconds%1)*1000)
    return "%02d:%02d:%02d,%03d"%(sHours,sMinutes,sSeconds,sMilliseconds)

def timestamp2second(timestamp):
    m=re.match("(\d\d):(\d\d):(\d\d),(\d\d\d)",timestamp)
    if m:
        seconds=(int(m.group(1))*60+int(m.group(2)))*60+int(m.group(3))+float(m.group(4))/1000
        return seconds
    else:
        print("problème de timestamp")
        return

def shiftTime(timestamp,decal):
    return second2timestamp(timestamp2second(timestamp)+decal)

In [21]:
punct = list('''!()[]{};:-'"\,<>./?@#$%^&*_~''')
maxSize=8

def removePunct(texte):
    result=texte
    for p in punct:
        result=result.replace(p,"")
    return result

### Gestion des structures JSON de Whisper

In [22]:
def getWords(whisper):
    lWords=[]
    for segment in whisper["segments"]:
        for word in segment["words"]:
            lWords.append((second2timestamp(word["start"]),
                              second2timestamp(word["end"]),
                              word["text"],
                              word["confidence"]))        
    return [(t1,t2,w,c) for t1,t2,w,c in lWords if w!=""]

def getList(tWhisper):
    return [unidecode.unidecode(removePunct(w)).lower() for _,_,w,_ in tWhisper]

In [23]:
srt=getWords(whisper1)
srtList=getList(srt)

### Trouver la jointure

In [24]:
def findSample(sample,ref):
    i=0
    while i<len(ref)-len(sample):
        # print("ref",i)
        if sample==ref[i:i+len(sample)]:
            return i
        else:
            i+=1
    return None

### Collage des mots
1. on cherche la partie commune entre le nouveau segment et les sous-titres précédents
2. on peut régler la longueur de la partie commune pour optimiser le collage *maxSize*
3. on incrémente les temps de la nouvelle partie à partir du deuxième mot (pour éviter les erreurs de timestamp en début d'enregistrement)

In [25]:
def pasteWords(srt,whisper2):
    srtList=getList(srt)
    w2=getWords(whisper2)
    w2List=getList(w2)
    lenW2=len(w2)
    compSize=min(maxSize,lenW2)
    i=1
    print(compSize)
    while i<lenW2-compSize:
        sample=w2List[i:i+compSize]
        ref=srtList
        # print(sample)
        idx=findSample(sample,ref)
        if idx:
            print("found",i,idx)
            break
        else:
            i+=1
    if idx:
        print("found",i,idx,srtList[idx],srt[idx][0],w2List[i],w2[i][0])
        tDecal=timestamp2second(srt[idx+1][0])-timestamp2second(w2[i+1][0])
        print(second2timestamp(tDecal))
        newSrt=[]
        for n in range(i+1,lenW2):
            nWord=w2[n]
            newWord=(shiftTime(nWord[0],tDecal),shiftTime(nWord[1],tDecal),nWord[2],nWord[3])
            newSrt.append(newWord)
        print(srt[idx+1],newSrt[0])
        return srt[:idx]+newSrt
    else:
        return None

In [26]:
if segments:
    for i in range(1,segments):
        print(i)
        json2=repJson+"extrait-%02d.wav.json"%i
        with open(json2) as inFile:
            whisper2=json.load(inFile)
        newSrt=pasteWords(srt,whisper2)
        if not newSrt:
            break
        else:
            srt=newSrt
# print(srt)

1
8
found 1 165
found 1 165 soit 00:01:00,159 soit 00:00:00,180
00:00:59,920
('00:01:00,280', '00:01:00,979', 'très', 0.994) ('00:01:00,280', '00:01:00,920', 'très', 0.962)
2
8
found 1 377
found 1 377 un 00:02:00,560 un 00:00:00,440
00:02:00,019
('00:02:00,700', '00:02:00,778', 'peu', 1.0) ('00:02:00,700', '00:02:00,859', 'peu', 0.99)
3
8
found 16 528
found 16 528 et 00:03:24,098 et 00:00:24,179
00:03:00,000
('00:03:24,399', '00:03:24,519', 'on', 0.995) ('00:03:24,399', '00:03:24,538', 'on', 0.988)
4
8
found 20 645
found 20 645 oh 00:04:14,718 oh 00:00:15,019
00:03:59,998
('00:04:15,079', '00:04:15,259', 'là', 0.802) ('00:04:15,079', '00:04:15,257', 'là', 0.901)
5
8
found 1 769
found 1 769 penalite 00:04:59,978 penalite 00:00:00,100
00:04:59,859
('00:05:00,379', '00:05:00,398', 'partout.', 0.984) ('00:05:00,379', '00:05:00,619', 'partout.', 0.681)
6
8
found 1 957
found 1 957 rythme 00:05:59,859 rythme 00:00:00,100
00:05:59,858
('00:06:00,258', '00:06:00,319', 'de', 0.963) ('00:06:00,25

In [27]:
pause=0.200

srtTours=[]
srtMots=[]
bTour=True
prevEnd=0
pStart=second2timestamp(0)
pEnd=0

for i,(start,end,mot,confidence) in enumerate(srt):
    # print(start,end,mot,confidence)
    tStart=timestamp2second(start)
    tEnd=timestamp2second(end)
    # print(start,tStart,prevEnd)
    if tStart-prevEnd>pause and prevEnd!=0:
        pEnd=second2timestamp(prevEnd)
        pTour=" ".join(srtMots)
        srtTours.append((pStart,pEnd,pTour))
        pStart=start
        prevEnd=tEnd
        srtMots=[mot]
        bPhrase=True
    else:
        srtMots.append(mot)
        prevEnd=tEnd
pTour=" ".join(srtMots)
srtTours.append((pStart,pEnd,pTour))

In [28]:
srtLines=[]
for i,(start,end,mot) in enumerate(srtTours):
    print(start,end,mot)
    n=i+1
    srtLines.append(str(n))
    srtLines.append(start+" --> "+end)
    srtLines.append(mot)
    srtLines.append("")


00:00:00,000 00:00:13,618 [*] La [*] finale, la grande finale de la catégorie des moins de [*] 78 kg avec Audrey Tcheméo [*] qui mène [*] 4 à 2 face à Stenuys. Stenuys, [*] c'est tout [*] simplement la [*] numéro 1 mondiale.
00:00:14,560 00:00:19,260 Et le public est chaud, bien sûr, [*] parce qu'Audrey, c'est toujours aussi du spectacle.
00:00:22,039 00:00:25,018 Et elle est chez elle. Ouh là là ! Elle est chez elle et déterminée, la française.
00:00:26,739 00:00:29,640 Eh oui, l'œil du tigre d'Audrey, ou plutôt la lionne indomptable.
00:00:30,000 00:00:39,859 [*] Eh [*] oui, accrochez vos ceintures parce que là, [*] c'est du grand Audrey Tcheméo.
00:00:40,259 00:00:52,618 Elle est ultra motivée. [*] Remontée à bloc, Lucie Descosse, la championne olympique, devenue coach [*] sur ses talons. Et Audrey qui se frappe le visage, [*] qui rentre [*] sur le tatami, prête à en découdre.
00:00:53,380 00:01:03,039 Il va falloir être fort du côté de Stenuys parce que [*] c'est un lion enragé qui

In [29]:
with codecs.open(json1.replace(".json","-tours.srt"),"w",encoding="utf8") as outFile:
    for srtLine in srtLines:
        outFile.write(srtLine+"\n")